Coisas que falta escrever:

characteristics of the selected algorithms

parameters employed in the methods

how the chosen methods work with the nature of the action-space representation

on-policy x off-policy results

did you achieve the policy?

non-linear function approximation advantages and disadvantages in relation to project 1

# Objetivos do Trabalho
O objetivo deste trabalho é aplicar métodos de aprendizado por reforço ao problema Snake, sendo estes métodos um on-policy e um off-policy, a fim de avaliá-los quanto à qualidade das soluções encontradas e custo computacional. Também pretende-se comparar os resultados obtidos com estes novos métodos aos resultados obtidos previamente no Projeto 1.

Para o projeto 2, mantivemos o mesmo problema do projeto 1 (o jogo Snake) e a mesma formulação MDP. Assim, a natureza do ambiente e os estados terminais são os mesmos do projeto 1. Também mantivemos as mesmas funções de reforço experimentadas no projeto 1. Dessa forma, o que muda em relação ao projeto 1 são os algoritmos empregados para resolver o problema. Realizamos experimentos com os métodos DQN (Deep Q-Learning) e A2C (Advantage Actor-Critic).

# Membros do Grupo e Contribuições

### Ana Clara Zoppi Serpa (RA 165880)
- Experimentos com DQN
- Análise de resultados experimentais
- Escrita do relatório ()
- Gravação do vídeo

### Gabriel Oliveira dos Santos (RA 197460)
- Experimentos com A2C
- Análise de resultados experimentais
- Escrita do relatório (explicação do A2C)
- Adaptação do nosso embiente para usar a biblioteca Stable Baselines

### Silvio Bento Garcia Junior (RA 265194)
- Experimentos com PPO
- Escrita do relatório (explicação do DQN)
- Análise de resultados experimentais

### Tito Barbosa Rezende (RA 025327)
- Experimentos com DQN
- Escrita do relatório (explicação do DQN)
- Análise de resultados experimentais

### Matteo Di Fabio (RA 264339)
- Experimentos com A2C
- Adaptação do nosso embiente para usar a biblioteca Stable Baselines
- Análise de resultados experimentais

## Link para o vídeo

# Problema
Escolhemos trabalhar com o jogo Snake, também conhecido como "Serpente" ou "Jogo da Cobrinha", disponível em https://www.google.com/fbx?fbx=snake_arcade.

O jogador é uma cobra, inicialmente pequena, e seu objetivo é comer maçãs. A cobra pode se mover para cima, para baixo, para a esquerda e para a direita. Maçãs aparecem em posições aleatórias do mapa, e o jogador deve se movimentar a fim de passar pela posição que contém a maçã atual. O jogador pode se mover livremente, mas não pode colidir com as bordas do mapa (paredes), nem com seu próprio corpo. Conforme as maçãs são consumidas, o tamanho da cobra aumenta, e há mais risco de colidir com seu próprio corpo ao se movimentar pelo mapa, assim aumentando a dificuldade do jogo.

## Formulação do problema como MDP

Conforme visto em aula, um Processo Decisório de Markov (MDP) é uma tupla <S, P, A, R, gamma>, sendo S um conjunto finito de estados, A um conjunto finito de ações, P uma matriz de probabilidades de transições entre estados, R uma função de reforço e gamma um fator de desconto, que é um número real no intervalo [0,1].

Um estado é uma tupla com 11 valores booleanos:
1. Se há perigo (parede ou o próprio corpo da cobra) uma ou duas unidades do tabuleiro à frente;
2. Se há perigo uma ou duas unidades à direita;
3. Se há perigo uma ou duas unidades à esquerda;
4. Se a cobra está se movendo para a esquerda;
5. Se a cobra está se movendo para a direita;
6. Se a cobra está se movendo para cima;
7. Se a cobra está se movendo para baixo.
8. Se há comida à esquerda;
9. Se há comida à direita;
10. Se há comida para cima;
11. Se há comida para baixo;

Há 2^11 estados possíveis.

Essas direções (esquerda, direita, cima, baixo) são sempre consideradas na perspectiva da cabeça da cobra.

As ações possíveis são:
- Continuar indo na mesma direção, representada pelo vetor [1,0,0];
- Virar para a direita, representada por [0,1,0];
- Virar para a esquerda, representada por [0,0,1];

A tela de jogo tem 440 pixels de largura e 440 pixels de altura. Cada parte do corpo da cobra é um quadrado de 20 pixels de altura e 20 pixels de largura. A maçã tem 20 pixels de altura e 20 pixels de largura também.

A matriz de transição de probabilidades P contém somente uns e zeros, pois dada uma posição (estado) atual no tabuleiro e uma ação, o resultado é sempre o mesmo. Para cada estado atual **s**, estado resultante **s'** quando tomada uma ação **a** e estados restantes **Z** que não são obtidos tomando **a** a partir de **s**, a probabilidade de ir de **s** para **s'** é 1, enquanto que de **s** a qualquer dos **Z** é zero. Isso vale para todos os estados e ações, por isso a matriz terá somente zeros e uns.

Quanto à função de reforço, definimos uma função padrão e depois exploramos outras. Isso será detalhado mais adiante no relatório, assim como os diferentes fatores de desconto (gamma) adotados nos experimentos.

## Natureza do ambiente

* Determinístico ou estocástico: **determinístico**, pois, dado um estado corrente e uma ação, o estado resultante é sempre o mesmo (dada uma posição, a ação de continuar na mesma direção/virar para a direita/virar para a esquerda leva sempre ao mesmo destino).

* Contínuo ou discreto: **discreto**, já que temos um conjunto finito de ações que podemos escolher tomar a partir de um certo estado (virar para a direita, virar para a esquerda, continuar na mesma direção).

* Episódico ou não episódico: **episódico**, sendo cada episódio um jogo. O episódio termina somente quando a cobra morre.

* Single-agent ou multi-agent: **single-agent**, pois nosso único agente é a cobra.

## Modelo de discretização adotado

Como nosso ambiente já é discreto, não precisamos adotar nenhuma estratégia de discretização.

## Estados Terminais

Nossos estados terminais são aqueles referentes à morte da cobra, portanto ocorrem somente auando a cobra colide com a parede e quando a cobra colide com seu próprio corpo.

## Funções de Reforço

Nossa **função de reforço padrão** foi -50 quando a cobra morre, +10 quando come a maçã e 0 caso contrário.

```
def default_reward(env):
    """
    Return the reward.
    The reward is:
        -50 when Snake crashes.
        +10 when Snake eats food
        0 otherwise
    """
    reward = 0
    if env.game.crash:
        reward = -10
    elif env.player.eaten:
        reward = 10

    return reward
```
No projeto 1, experimentamos outras funções de reforço. No entanto, para os experimentos do projeto 2, decidimos manter sempre a `default_reward`, tendo em vista que apresentou bons resultados no projeto 1.

# Pontos-chave da implementação

- falta escrever com uns pedaços importantes do código aqui

## Particularidades e restrições da implementação

Para executar o código, é preciso instalar a biblioteca PyGame. Isso pode ser feito com o comando `pip install pygame` no terminal. Também é preciso instalar a biblioteca Stable Baselines 3. Isso pode ser feito com o comando `pip install stable-baselines3`.

# Métodos

## DQN (Deep Q-Learning)

### Ideia geral do método

O aprendizado por reforço pode ser suficientemente aplicável ao ambiente onde todos os estados alcançáveis podem ser gerenciados e armazenados na memória RAM padrão do computador. No entanto, no ambiente em que o número de estados supera a capacidade dos computadores, a abordagem padrão de RL não é muito aplicável. Além disso, em ambiente real, o agente tem que enfrentar problemas de estados contínuos, variáveis contínuas e de controle contínuo.

Tendo em mente a complexidade do ambiente em que o agente deve operar, a tabela Q de estados ações usada no RL é substituída por uma Rede Neural Profunda que mapeia o ambiente (aproximação não linear) para ações do agente. A arquitetura da rede, a escolha dos hiperparâmetros da rede e o aprendizado são realizados durante a fase de treinamento. O DQN permite que o agente explore ambientes não estruturados e adquira conhecimentos que, com o tempo, possibilitem a imitação do comportamento humano.

Durante o processo de treinamento, o agente interage com o ambiente e recebe dados, que são utilizados durante o aprendizado da Q-network. O agente explora o ambiente para construir uma imagem completa das transições e resultados da ação. No início, o agente decide sobre as ações de forma aleatória. Enquanto explora o ambiente, o agente tenta olhar para a rede Q (aproximação) para decidir como agir.

É utilizado um método epsilon-guloso. Para tornar o processo de treinamento mais estável, aplica-se um buffer de replay que memoriza experiências do comportamento do agente. Em seguida, o treinamento é realizado em amostras aleatórias do buffer.

### Pseudocódigo
![](DQN.png)

### Passos do Algoritimo
1. Inicializar o buffer de replay.

2. Pré-processar o ambiente e passe o estado S como entrada para a rede neural, que retornará os valores Q para cada ação possível a partir do estado S.

3. Selecionar uma ação com a política epsilon-greedy, ou seja: com probabilidade epsion, selecionamos uma ação aleatória A e, com probabilidade 1-epsilon, selecionamos uma ação A para a qual Q é máximo.

4. Tendo escolhido A, o agente realiza essa ação a partir de S para chegar em S' e receber uma recompensa R.

5. Guardar essa transição no buffer de replay (<S,A,R,S’>).

6. Amostrar alguns batches aleatórios de transições do replay buffer e calcular a loss.

7. Descida do gradiente para minimizar loss da rede neural.

8. Após k passos, atualizar os pesos da rede alvo com os pesos da rede real.

9. Realizar esses passos por M episódios. 

### Parâmetros

A biblioteca Stable Baselines (https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html) permite alterar diversos parâmetros do método DQN. Neste trabalho, exploramos:

- **total_timesteps**: o número total de amostras, ou seja, de passos do ambiente ("environment steps") que serão usados para o treinamento.
- **learning_starts**: a partir de qual passo o aprendizado deve começar. Em outras palavras, são coletadas learning_steps transições (passos) do modelo antes de começar de fato o treinamento.
- **gamma**: fator de desconto para recompensas futuras. Este fator regula o tipo de atitude do agente em relação a recompensas de longo prazo - ele pode priorizar recompensas futuras (quanto mais próximo de 1 gamma estiver) ou priorizar recompensas imediatas (quanto mais próximo de 0 gamma estiver).
- **exploration_fraction**: fração do período de treinamento na qual a taxa de exploração, i.e, escolha de ações aleatórias, é diminuída.
- **learning_rate**: taxa de aprendizado.
- **batch_size**: tamanho do batch para cada atualização de gradiente.


## A2C (Advantage Actor-Critic)

### Parâmetros

A biblioteca Stable Baselines (https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html) permite alterar diversos parâmetros do método A2C. Neste trabalho, exploramos:

- **total_timesteps**: o número total de amostras, ou seja, de passos do ambiente ("environment steps") que serão usados para o treinamento.
- **gamma**: fator de desconto para recompensas futuras. Este fator regula o tipo de atitude do agente em relação a recompensas de longo prazo - ele pode priorizar recompensas futuras (quanto mais próximo de 1 gamma estiver) ou priorizar recompensas imediatas (quanto mais próximo de 0 gamma estiver).
- **learning_rate**: taxa de aprendizado.
- **normalize_advantage**: normalizar (true) ou não (false) a vantagem.
- **ent_coef**: coeficiente de entropia para o cálculo da loss.
- **vf_coef**: coeficiente da função valor para o cálculo da loss.
- **use_rms_prop**: utilizar o otimizador RMSprop (true) ou utilizar o otimizador Adam (false).
- **n_eval_episode**: número de episódios para avaliar o agente.


# Discussão de Resultados

## Método DQN e seus parâmetros

## Método A2C e seus parâmetros

## Comparando DQN e A2C

## Comparando DQN com resultados do projeto 1

## Comparando A2C com resultados do projeto 1

## Principais Conclusões

# Observações

## Notebooks auxiliares, código completo e planilha de experimentos

Aqui neste notebook, colocamos exemplos de código para os principais experimentos. No entanto, todos os nossos experimentos estão em notebooks auxiliares usados por cada membro do grupo.

- Notebook auxiliar 1 - experimentos com DQN realizados por Ana Clara Zoppi Serpa: https://github.com/AnaClaraZoppiSerpa/snake/blob/main/PROJETO2/notebooks/ana_DQN.ipynb
- Notebook auxiliar 2 - experimentos com A2C realizados por Gabriel Oliveira dos Santos: https://github.com/AnaClaraZoppiSerpa/snake/blob/main/PROJETO2/notebooks/gabriel.ipynb
- Notebook auxiliar 3 - experimentos com DQN realizados por Tito Barbosa Rezende: https://github.com/AnaClaraZoppiSerpa/snake/blob/main/PROJETO2/notebooks/tito_DQN.ipynb
- Notebook auxiliar 4 - experimentos com A2C realizados por Matteo Di Fabio: https://github.com/AnaClaraZoppiSerpa/snake/blob/main/PROJETO2/notebooks/Matteo.ipynb

Além disso, todos os resultados estão disponíveis na seguinte planilha: https://docs.google.com/spreadsheets/d/1MHTG_k1Mc4TGba2h3Fi95k1qa0V8B3bf4FI8xGXIzMg/edit?usp=sharing

O código completo está disponível no GitHub: https://github.com/AnaClaraZoppiSerpa/snake

## Outras

Tentamos realizar experimentos com o método PPO (Proximal Policy Optimization), mas não obtivemos bons resultados.


# Referências

- Aulas da professora, disponibilizadas no Google Classroom da disciplina.

- Livro Reinforcement Learning: An Introduction, Sutton and Barto, 2a Edição.

- Mnih, Volodymyr, et al. "Asynchronous methods for deep reinforcement learning." International conference on machine learning. PMLR, 2016.

- Documentação da biblioteca Stable Baselines, método DQN: https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html

- Documentação da biblioteca Stable Baselines, método A2C: https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html

- Explicação sobre Deep Q-Learning: https://en.m.wikipedia.org/wiki/Q-learning#Deep_Q-learning

- Artigo sobre métodos actor-critic: https://towardsdatascience.com/understanding-actor-critic-methods-931b97b6df3f

- Mais sobre Deep Q-Learning: https://medium.com/@markus.x.buchholz/deep-reinforcement-learning-introduction-deep-q-network-dqn-algorithm-fb74bf4d6862

- Mnih, V., Kavukcuoglu, K., Silver, D. et al. Human-level control through deep reinforcement learning. Nature 518, 529–533 (2015). https://doi.org/10.1038/nature14236

### Artigos e exemplos de implementação

- Artigo sobre Snake com Deep Q-Learning: https://towardsdatascience.com/how-to-teach-an-ai-to-play-games-deep-reinforcement-learning-28f9b920440a

- Repositório do GitHub com Snake e Deep Q-Learning, no qual nos baseamos inicialmente para realizar o projeto: https://github.com/maurock/snake-ga

- Exemplo de implementação do SARSA: https://www.geeksforgeeks.org/expected-sarsa-in-reinforcement-learning/

- Exemplo de implementação de Environment com Gym: https://github.com/pedrohbtp/snake-rl/blob/master/snake_rl.ipynb

- Tutorial de Reinforcement Learning: https://araffin.github.io/slides/rl-tuto-jnrr19/#/4

- Artigo sobre SARSA(lambda) para vários jogos, inclusive Snake: http://cs229.stanford.edu/proj2012/JohnsonRobertsFisher-LearningToPlay2DVideoGames.pdf

- Exemplo de implementação de Aproximador de Função: https://github.com/metastableB/Naagin-Naggin/blob/master/dlsnake/agents/approxQAgent.py

- Exemplos de implementações de SARSA e Monte Carlo: https://github.com/ralhadeff/machine-learning-tools/tree/master/ReinforcementLearning

- Exemplos de implementações baseados no livro Reinforcement Learning: An Introduction: https://github.com/flyywh/reinforcement-learning-1

- Artigos sobre Snake, Q-Learning e SARSA: https://dkdennis.xyz/static/Nagging-report.pdf, http://cs229.stanford.edu/proj2016spr/report/060.pdf

### Outras páginas

- https://towardsdatascience.com/reinforcement-learning-rl-101-with-python-e1aa0d37d43b

- https://towardsdatascience.com/slitherin-solving-the-classic-game-of-snake-with-ai-part-2-general-purpose-random-monte-25dc0dd4c4cf

- https://towardsdatascience.com/function-approximation-in-reinforcement-learning-85a4864d566

- https://towardsdatascience.com/monte-carlo-learning-b83f75233f92

- https://medium.com/reinforcement-learning-a-step-by-step-implementati/reinforcement-learning-a-step-by-step-implementation-using-sarsa-1cfd3e64775a

- https://www.programmersought.com/article/94202345056/

- https://www.youtube.com/watch?v=l0sFUU7vScA